<a href="https://colab.research.google.com/github/DongUk-Park/RnD/blob/main/POI/NAIS/NAIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
import csv
import numpy as np
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import time

## Data Utils

In [27]:
def load_data():
  train_dataset_path = '/content/history_train.csv'
  validation_dataset_path = '/content/history_validation.csv'
  test_dataset_path = '/content/history_test.csv'

  train_history_data = []
  validation_history_data = []
  test_history_data = []

  with open(train_dataset_path, 'r', newline='') as csv_train:
      csv_reader = csv.reader(csv_train)
      header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
      header = [int(item) for item in header]
      train_history_data.append(header)
      for row in csv_reader:
          r = [int(item) for item in row]
          train_history_data.append(r)

  with open(validation_dataset_path, 'r', newline='') as csv_validation:
      csv_reader = csv.reader(csv_validation)
      header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
      header = [int(item) for item in header]
      validation_history_data.append(header)
      for row in csv_reader:
          r = [int(item) for item in row]
          validation_history_data.append(r)

  with open(test_dataset_path, 'r', newline='') as csv_test:
      csv_reader = csv.reader(csv_test)
      header = next(csv_reader)  # 헤더 행, 실제론 data 바로 들어옴
      header = [int(item) for item in header]
      test_history_data.append(header)
      for row in csv_reader:
          r = [int(item) for item in row]
          test_history_data.append(r)
  business_id_info = pd.read_csv('/content/business_info_in_philadelphia.csv')
  item_list = business_id_info.index.tolist()


  # print("여기는 함수한")
  # print(business_id_info)
  # print(item_list)
  # print(train_history_data[0])

  num_users = len(train_history_data) # 필라델피아의 사용자 수 : 15919명
  num_items = business_id_info.shape[0] # 필라델피아의 가게 수 : 14603개 , 마지막 business_id : 150336
  return item_list, num_users, num_items, train_history_data, validation_history_data, test_history_data

In [10]:
def get_train_data_user(item_list, num_users, train_list, num_negatives):
    """
    학습데이터를 확인하고 num_negatives만큼의 negative데이터를 뽑아내 학습에 사용
    negative sample
    """
    num_items = len(item_list)
    user_input, item_input, labels, batch_length = [],[],[],[]

    for u in range(num_users):
       np.random.shuffle(train_list[u])
       if u == 0:
           batch_length.append((1+num_negatives) * len(train_list[u]))
       else:
           batch_length.append((1+num_negatives) * len(train_list[u])+batch_length[u-1])

       for i in train_list[u]:
            # positive instance
            user_input.append(u)
            item_input.append(i)
            labels.append(1)
            # negative instances
            for t in range(num_negatives):
                j_idx = np.random.randint(0, num_items) # num_items == len(item_list), item_list에서 Negative sample을 골라야하기 때문에 인덱스를 골라줌
                j = item_list[j_idx]
                while j in train_list[u]:
                    j_idx = np.random.randint(0, num_items)
                    j = item_list[j_idx]
                user_input.append(u)
                item_input.append(j)
                labels.append(0)
    return user_input, item_input, labels, batch_length

def get_input_data(batch_length, train_list, user_input, item_input, batch):
    """
    recall, precision을 위한 입력 데이터 생성
    """
    p=0
    target_idx = []
    if batch == 0:
        p = 0
    else:
        p = batch_length[batch-1] # p : 지난 배치를 지나감

    target_idx = item_input[p:batch_length[batch]]

    temp = set(train_list[user_input[p]])

    history = []
    for idx in range(batch_length[batch]-p):
        history.append(list(temp))
    p=batch_length[batch]

    return history, target_idx


## Evaluate

In [28]:
def eval(model, num_users, testPositives, history_list, target_idx_list):
    model.eval()
    hit = [0,0,0,0,0]
    precision = [0,0,0,0,0]
    recall = [0,0,0,0,0]

    k=[10,20,30,40,50]
    a = 0.9

    with torch.no_grad():
        for u in range(num_users):
            count = 0
            history = history_list[u]
            target_idx = target_idx_list[u]
            results=model(history, target_idx)

            for i in range(len(k)):
                topk = torch.topk(results,k[i])
                topSet = set(target_idx[topk.indices.tolist()])

                positives = set(testPositives[u])
                count = len(topSet & positives)

                precision[i] += (count/k[i])
                recall[i] += (count/len(testPositives[u]))
                if count >0:
                    hit[i] += 1

        for i in range(len(k)):
            recall[i]/= num_users
            precision[i]/= num_users
            hit[i]/= num_users

    return recall , precision, hit


## Train

In [44]:
def train(model, epoch, history, user_input, item_input, labels, batch_length, optimizer, validation_history_data, user_input_validation, item_input_validation, labels_validation, batch_length_validation, test_history_data):
    loss_func = nn.BCELoss()
    before_loss = 210000000.0
    max_re = 0.0

    recall = []
    prec = []
    hit = []

    for e in range(epoch):
        model.train()
        train_loss = 0.0
        start_time = int(time.time())

        p = 0 # 지난 배치는 지나가도록 확인해주는 변수

        for b in range(len(batch_length)): # user 수 만큼 반복
          print(f"User : {b}")
          batch_size = batch_length[b]
          user = []
          for i in range(5): # 5 : negative_num + 1
            user.extend(history[b])

          #print(f"user_len : {len(user)}")
          #print(f"p: {p}, batch_size : {batch_size}")
          item = item_input[p:batch_size]

          optimizer.zero_grad()
          prediction = model(user, item)
          temp = torch.tensor(labels[p:batch_size], dtype=torch.float32)
          loss = model.loss_func(prediction,temp)

          p = batch_size # p 업데이트
          loss.backward()
          train_loss += loss.item()
          optimizer.step()

        end_time = int(time.time())

        print("Train Epoch: {}; time: {} sec; loss: {:.4f}".format(e+1, end_time-start_time,train_loss))

        re,pr,hi = eval(model, num_users, validation_history_data, train_history_data, item_input_validation)
        recall.append(re)
        prec.append(pr)
        hit.append(hi)

        print("recall: {} precision: {} hit: {};".format( re[0],pr[0],hi[0]))
        re,pr,hi,re_d,pr_d,hi_d = eval(model,num_users, test_history_data, train_history_data, item_input_test)

        recall.append(re)
        prec.append(pr)
        hit.append(hi)


        end_time = int(time.time())
        print("time: {} sec;".format( end_time-start_time))

    return recall , prec, hit


## NAIS Model Define

In [45]:
class NAIS_basic(nn.Module):
    def __init__(self, item_num, embed_size, beta): # embed_size : 64, beta : 0.5
        super(NAIS_basic, self).__init__()
        self.embed_size = embed_size # concat 연산 시 * 2
        self.item_num = item_num
        self.beta = beta

        self.embed_history = nn.Embedding(item_num, embed_size) # (m:14603 * d:64), 과거 방문한 데이터(q), 유저별로 각각 하나씩 가져야하나 ?
        self.embed_target = nn.Embedding(item_num, embed_size) # (m:14603 * d:64), 예측 데이터(p)


        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.loss_func = nn.BCELoss() # binary cross entropy

        # Attention을 위한 MLP Layer 생성
        self.attn_layer1 = nn.Linear(embed_size, embed_size)
        self.attn_layer2 = nn.Linear(embed_size, 1, bias = False)

        self._init_weight_()

    def _init_weight_(self):
        # weight 초기화, 표준편차 : 0.01
        nn.init.normal_(self.embed_history.weight, std=0.01)
        nn.init.normal_(self.embed_target.weight, std=0.01)

        # bias 초기화
        for m in self.modules():
            if isinstance(m, nn.Linear) and m.bias is not None:
                m.bias.data.zero_()

    def forward(self, history, target):
        #배치 사이즈만큼 잘라서 넣어줌
        #print(len(history), len(target))
        history_idx = torch.LongTensor(history)
        target_idx = torch.LongTensor(target)
        prediction = self.attention_network(history_idx,target_idx)
        return self.relu(prediction)

    def attention_network(self, user_history, target_item):
        """
        b: batch size
        n: history size
        d: embedding size
        """
        #print("#### def attention network start #####")
        #print(len(user_history), user_history)
        #print(len(target_item), target_item)

        history = self.embed_history(user_history) # (b * n * d)
        target = self.embed_target(target_item) # (b * 1 * d)
        batch_dim = len(target)

        #print("#### embedding finished")
        #print(history.shape)
        #print(target.shape)
        #print(batch_dim)

        target = torch.reshape(target,(batch_dim, 1,-1))
        input = history * target # (b * n * d)


        #print("#### attention layer start")
        attention_result = self.relu(self.attn_layer1(input)) # (n * d)
        attention_result = self.attn_layer2(attention_result) # (n * 1)

        exp_A = torch.exp(attention_result) # (b * n * 1)
        exp_A = exp_A.squeeze(dim=-1)# (b * n)

        mask = self.get_mask(user_history,target_item)
        exp_A = exp_A * mask
        exp_sum = torch.sum(exp_A,dim=-1) # (b * 1)
        exp_sum = torch.pow(exp_sum, self.beta) # (b * 1)

        attn_weights = torch.divide(exp_A.T,exp_sum).T # (b * n)
        attn_weights = attn_weights.reshape([batch_dim,-1, 1])# (b * n * 1)
        result = history * attn_weights# (b * n * d)
        target = target.reshape([batch_dim,-1,1]) # (b * d * 1)

        prediction = torch.bmm(result, target).squeeze(dim=-1) # (b * n * 1) -> (b * n)
        prediction = torch.sum(prediction, dim = -1) # (b)
        #print(f"#### return prediction{prediction.shape}")
        return prediction

    def get_mask(self, user_history, target_item):
        target_item = target_item.reshape([len(target_item),1])
        mask = user_history != target_item
        return mask
    def loss_function(self, prediction, label):
        return self.loss_func(prediction, label)

## Main

In [ ]:
if __name__ == "__main__":
    # USE_CUDA = torch.cuda.is_available()
    # DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
    #print("사용하는 Device : ", DEVICE)
    epoch = 40
    num_negatives = 4

    item_list, num_users, num_items, train_history_data, validation_history_data, test_history_data = load_data()
    user_input_train, item_input_train, labels_train, batch_length_train = get_train_data_user(item_list, num_users, train_history_data, num_negatives)
    user_input_validation, item_input_validation, labels_validation, batch_length_validation = get_train_data_user(item_list, num_users, validation_history_data, num_negatives)
    user_input_test, item_input_test, labels_test, batchlength_test = get_train_data_user(item_list, num_users, test_history_data, num_negatives)
    print(num_items, num_users)
    print(len(item_list), item_list[0], item_list[-1])

    for i in range(5):
        nais_model_name = "NAIS_basic_product"

        nais_model = NAIS_basic(num_items, 64, 0.5) # num_items, embed_size, beta
        optimizer1 = optim.Adagrad(nais_model.parameters(), lr=0.005, weight_decay=0.001)
        print(nais_model_name + " model")

        recall, prec, hit, recall_d, prec_d, hit_d = train(nais_model, epoch, train_history_data, user_input_train, item_input_train, labels_train, batch_length_train, optimizer1, validation_history_data, user_input_validation, item_input_validation, labels_validation, batch_length_validation, test_history_data)

        with open("/content/result/"+nais_model_name+ str(i)+".txt", mode='wt') as f:
          for g in range(len(recall)):
              f.write("epoch: "+str((g+1))+"-----------------------------------------\n")
              f.write("recall:"+ str( recall[g])+", precision: "+ str(prec[g])+", hit: "+ str(hit[g])+"\n")


14585 15919
14585 0 14584
NAIS_basic_product model
User : 0
User : 1
User : 2
User : 3
User : 4
User : 5
User : 6
User : 7
User : 8
User : 9
User : 10
User : 11
User : 12
User : 13
User : 14
User : 15
User : 16
User : 17
User : 18
User : 19
User : 20
User : 21
User : 22
User : 23
User : 24
User : 25
User : 26
User : 27
User : 28
User : 29
User : 30
User : 31
User : 32
User : 33
User : 34
User : 35
User : 36
User : 37
User : 38
User : 39
User : 40
User : 41
User : 42
User : 43
User : 44
User : 45
User : 46
User : 47
User : 48
User : 49
User : 50
User : 51
User : 52
User : 53
User : 54
User : 55
User : 56
User : 57
User : 58
User : 59
User : 60
User : 61
User : 62
User : 63
User : 64
User : 65
User : 66
User : 67
User : 68
User : 69
User : 70
User : 71
User : 72
User : 73
User : 74
User : 75
User : 76
User : 77
User : 78
User : 79
User : 80
User : 81
User : 82
User : 83
User : 84
User : 85
User : 86
User : 87
User : 88
User : 89
User : 90


In [20]:
t_li = [11471,  5473,  3206,  9111,  7140, 12063,  5755, 12509,  5629,  1838,
        10311,  3560,  4583, 14585,  3676,  8564,  3397, 14471,  7808,  9060,
         6653,  1037,  4198,  6694,  7696,  9982,  6613,  2436,  8214,  3028]
t_li.sort()
print(t_li)

[1037, 1838, 2436, 3028, 3206, 3397, 3560, 3676, 4198, 4583, 5473, 5629, 5755, 6613, 6653, 6694, 7140, 7696, 7808, 8214, 8564, 9060, 9111, 9982, 10311, 11471, 12063, 12509, 14471, 14585]
